# Task 1. Algorithm's analysis

In [3]:
import random
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, groupby

from networkx.algorithms import tree
from networkx.algorithms import floyd_warshall_predecessor_and_distance

import numpy.typing as npt

## Generating graph

In [4]:

# You can use this function to generate a random graph with 'num_of_nodes' nodes
# and 'completeness' probability of an edge between any two nodes
# If 'directed' is True, the graph will be directed
# If 'draw' is True, the graph will be drawn
def gnp_random_connected_graph(num_of_nodes: int,
                               completeness: int,
                               directed: bool = False,
                               draw: bool = False):
    """
    Generates a random graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted (in case of undirected graphs)
    """

    
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    edges = combinations(range(num_of_nodes), 2)
    G.add_nodes_from(range(num_of_nodes))
    
    for _, node_edges in groupby(edges, key = lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        if random.random() < 0.5:
            random_edge = random_edge[::-1]
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < completeness:
                G.add_edge(*e)
                
    for (u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(-5, 20)
                
    if draw: 
        plt.figure(figsize=(10,6))
        if directed:
            # draw with edge weights
            pos = nx.arf_layout(G)
            nx.draw(G,pos, node_color='lightblue', 
                    with_labels=True,
                    node_size=500, 
                    arrowsize=20, 
                    arrows=True)
            labels = nx.get_edge_attributes(G,'weight')
            nx.draw_networkx_edge_labels(G, pos,edge_labels=labels)
            
        else:
            nx.draw(G, node_color='lightblue', 
                with_labels=True, 
                node_size=500)
        
    return G

## Comparing algorithms

In [39]:
# This is a function we made to compare the execution times of different algorithms.

import timeit

def testing_algorithms(sizes, edge_probs, builtin_algorithm, custom_algorithm, number, directed=False):
    """
    Compare and visualize different algorithms

    :param sizes: list, A list of sizes of graphs on which we should test the algorithms
    :param edge_probs: float, The probability for extra edges to occur in every testing graph
    :param builtin_algorithm: callable, The built-in algorithm
    :param custom_algorithm: callable, The algorithm to compare the built-in to
    :param number: int, The number of times to run every algoritm
    """
    builtin_times = []
    custom_times = []
    
    for size in sizes:
        G = gnp_random_connected_graph(size, edge_probs, directed)
        
        builtin_time = timeit.timeit(lambda: builtin_algorithm(G), number=number)
        custom_time = timeit.timeit(lambda: custom_algorithm(G), number=number)
        
        builtin_times.append(builtin_time)
        custom_times.append(custom_time)
    
    plt.figure(figsize=(10, 6))
    plt.plot(sizes, builtin_times, label='Built-in algorithm', marker='x')
    plt.plot(sizes, custom_times, label='Our algorothm', marker='s')

    plt.xlabel('Graph Size (Number of Nodes)')
    plt.ylabel('Execution Time (Seconds)')
    plt.title('Performance Comparison of Algorithms')
    plt.legend()
    plt.grid()
    plt.show()

## Subtask 1.1: Prim's algorithm

### Built-in implementation

In [48]:
G = gnp_random_connected_graph(10, 0.5, False)

In [ ]:
mstp = tree.minimum_spanning_tree(G, algorithm="prim")

nx.draw(mstp, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

### Our implementation
Prim’s algorithm finds the Minimum Spanning Tree of a graph, meaning it connects all the nodes with the smallest total edge weight without forming a cycle.

How it works:

* Start with any node in the graph.
* Pick the smallest edge that connects the current tree to a new node.
* Add that node to the tree and mark it as visited.
* Repeat until all nodes are connected.

It keeps adding the cheapest possible edge that expands the tree while avoiding cycles. The result is a minimum-cost tree connecting all nodes.

### Time Complexity:
The algorithm runs in O(V²) time, where V is the number of nodes in the graph.

In [16]:
import heapq

def prim(original_graph: nx.Graph, starting_node: int = 0):
    mst = nx.Graph()
    mst.add_nodes_from(original_graph)
    min_heap = []

    def explore_node(node: int):
        mst.nodes[node]['visited'] = True
        for neighbor in original_graph.neighbors(node):
            if not mst.nodes[neighbor].get('visited', False):
                heapq.heappush(min_heap, (original_graph[node][neighbor]['weight'], node, neighbor))

    explore_node(starting_node)

    while min_heap:
        weight, node1, node2 = heapq.heappop(min_heap)

        if not mst.nodes[node2].get('visited', False):
            mst.add_edge(node1, node2, weight=weight)
            explore_node(node2)

    return mst

In [ ]:
mst = prim(G)

nx.draw(mst, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

### Comparing ours vs built-in

In [ ]:
# Here we specify the graph sizes on which we should test the algorithms.
sizes = [10, 25, 50, 75, 100]
edge_prob = 0.05

# The variable below is used to specify the number of how many times must each algorithm execute.
# Feel free to modify it if you feel like the testing takes too long.
number = 1000

# Running the testing
testing_algorithms(sizes, edge_prob, lambda G: tree.minimum_spanning_tree(G, algorithm="prim"), prim, number)

#### Excecution Times:
As expected, the built-in algorithm operates faster, although, interestingly enough, the lesser edge count gets, the faster operates our algorithm, even outracing the built-in one on values of edge_prob lesser than 0.05

#### Conclusion:
* Built-in algorithm is faster, obviously. I'm no such Python god.

## Subtask 1.2: Floyd-Warshall algorithm

### Built-in implementation

In [49]:
G = gnp_random_connected_graph(10, 0.5, True)

In [ ]:
# pred is a dictionary of predecessors, dist is a dictionary of distances dictionaries
try:
    pred, dist = floyd_warshall_predecessor_and_distance(G) 
    for k, v in dist.items():
        print(f"Distances with {k} source:", dict(v))
except:
    print("Negative cycle detected")

### Our implementation
The Floyd-Warshall algorithm finds the shortest paths between all pairs of nodes in a graph. 
It works by improving distance estimates using every node as a possible stop between two other nodes.

1. The algorithm stars by creating a distance dictionary where:
* The distance from a node to itself is 0.
* The distance between two directly connected nodes is set to the edge weight.
* The distance between all other nodes is set to infinity.
2. For each node, it checks if going through it makes the path between two other nodes shorter. If it is true, it updates the distance.
3. The algorithm repeats this process until all possible shorter paths are found.
4. the function also checks for a negative cycle, which is when the sum of weights in a loop is negative, leading to infinitely decreasing distances.
If `distance[node][node] < 0` for any node, the graph has a negative cycle.

### Time Complexity:
The algorithm runs in O(V³) time, where V is the number of nodes in the graph. This is because it checks all possible pairs of nodes for every intermediate node.

In [8]:
def floyd_warshall(G: nx.Graph) -> nx.Graph:
    """
    Calculates the shortest path distances between all pairs of nodes in the graph 
    using the Floyd-Warshall algorithm and returns a new graph with the shortest path 
    distances as edge weights
    args:
        original_graph (nx.Graph): an undirected graph represented as a NetworkX Graph
    returns:
        nx.Graph: a new graph where edges represent the shortest path distances between nodes
    """
    nodes = list(G.nodes)
    distance = {}
    for u in G.nodes:
        distance[u] = {}
        for v in G.nodes:
            distance[u][v] = float('inf')
    for u in nodes:
        distance[u][u] = 0

    edges = list(G.edges(data=True))
    for u, v, data in edges:
        weight = data.get('weight', 1)
        distance[u][v] = weight

    for k in nodes:
        for i in nodes:
            for j in nodes:
                if distance[i][k] != float('inf') and distance[k][j] != float('inf'):
                    if distance[i][j] > distance[i][k] + distance[k][j]:
                        distance[i][j] = distance[i][k] + distance[k][j]

    for node in nodes:
        if distance[node][node] < 0:
            pass

    return distance


In [ ]:
try:
    distance = floyd_warshall(G)
    for source, targets in distance.items():
        print(f"Distances with {source} source:", targets)
except Exception as e:
    print(e)

### Comparing ours vs built-in

In [ ]:
# Here we specify the graph sizes on which we should test the algorithms.
sizes = [10, 25, 50, 75, 100]
edge_prob = 0.5
# The variable below is used to specify the number of how many times must each algorithm execute.
# Feel free to modify it if you feel like the testing takes too long.
number = 100

# Running the testing
testing_algorithms(sizes, edge_prob, nx.floyd_warshall_predecessor_and_distance, floyd_warshall, number)

### Description

#### Execution Times:
For smaller graph sizes like 10 vertices, both our implementation and the built-in Floyd-Warshall have very similar execution times, with a difference of less than 1 second.

#### As the graph size increases:
 
* For 50 vertices, the time difference becomes more noticeable, with a bit more than 1 second difference
* For 100 vertices and more, the gap is much larger. For example, our implementation is taking over 20 seconds on 100 vertices, while the built-in algorithm takes less than 10 seconds, which is a big gap. 

#### Analysis of Differences:
At first the times are similar with a second difference less than a second.
For large graphs, however, the NetworkX function has to perform extra tasks like finding paths, it's designed to be more efficient and handles large graphs better.
Python can be slow when working with large amounts of data, which also slows down our implementation.


#### Where the biggest difference occurs:
The difference is most significant for larger graphs with 75 and 100 vertices. This is because our implementation's nested loops that slow down the proces with the increasing number of vertices. The built-in algorithm is more optimized for such larger graphs.

#### Conclusion:
* For small graphs, both algorithms are about the same.
* For larger graphs, the difference in execution time grows significantly. It is mainly because of the Python's slower execution of loops.

# Task 2: Decision Tree Classifier 

In [87]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## Description

### Goal:
Our goal in this project was to implement a decision tree classifier in Python to classify Iris flowers into three species: Iris Setosa, Iris Versicolor, and Iris Virginica. By using the Iris dataset, we aimed to understand how decision trees work in machine learning. 
The goal of the decision tree that we created is to split data into subsets based on the given values to make predictions on to which species each flower belongs. At each node of the tree, a decision is made to split the data into two groups, and this continues recursively until the tree is built.

### Dataset information:
The Iris dataset consists of 150 samples. Each sample belongs to one of three species. Each sample includes the following four features, measured in centimeters:

* Sepal Length
* Sepal Width
* Petal Length
* Petal Width
These features are an input for our decision tree model to distinguish between the three species.

### The process:
At each node, the algorithm decides which feature and what value to use to split the dataset into left and right. This decision is based on a measure of how  mixed the groups are.

The goal when splitting data is to create the purest possible subsets, meaning that the samples in each subset should mostly belong to the same species.
We used Gini impurity to measure the purity of the data at each node.

The tree-building process is recursive. At each step, the tree chooses the best feature to split the data and a value to divide the data into two groups. After splitting, the process repeats for each of the new groups. This continues until we reach a stopping point, like when the tree reaches its maximum depth or when the data can't be split any further.

The goal is to predict the species of a new Iris flower. When we get new data (like a new flower's measurements), the tree looks at the feature values and follows the path from the starting point (root) to the end (leaf) of the tree. At each step, it checks the feature values and moves left or right based on the value. When it reaches the end (a leaf node), it makes a prediction based on the most common species in that leaf.

## Setup

### Loading the dataset

In [ ]:
iris = load_iris()
X, y = iris.data, iris.target
X.shape, y.shape

### Train / test split

We train our model using training dataset and evaluate its performance basing on the test dataset. Reason to use two separate datasets is that our model learns its parameters from data, thus test set allows us to check its possibilities on completely new data.

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size= 0.20)
X_test.shape, y_test.shape

## Our implementation

### Node class

This is the class for nodes, and the whole tree is built from them. It contains methods for calculating the Gini impurity of a node and the heart of the whole algorithm - find_best_split() function. The tree is built in result of multiple executions of this function, which calculates the best condition for splitting a node.

In [90]:
class Node:
    """
    A class for nodes which together build up a decision tree.
    """

    def __init__(self, X: npt.NDArray, y: npt.NDArray, parent = None):
        """
        :param X: numpy array of form [[feature1,feature2, ... featureN], ...] (i.e. [[1.5, 5.4, 3.2, 9.8] , ...] for case with iris d.s.)
        :param y: numpy array of from [class1, class2, ...] (i.e. [0,1,1,2,1,0,...] for case with iris d.s.)
        """

        self.X = X
        self.y = y
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.predicted_classes, self.class_occurences = np.unique(y, return_counts=True)
        self.samples_count = len(y)
        self.gini = self.calculate_gini()
        self.parent = parent

    def calculate_gini(self):
        """
        The function for calculating the gini index of a node.
        """
        return 1 - sum((num / self.samples_count)**2 for num in self.class_occurences)

    def find_best_split(self):
        """
        The main function in the whole decision tree, ironically not belonging to its class. 
        Does everything: finds the best feature index and threshold for the node and returns the node's
        children. You can even say the build_tree() function is a mere wrapper for this one.
        """
        samples = list(zip(self.X, self.y))
        best_info_gain = -float("inf")

        for feature_index in range(self.X.shape[1]):
            possible_thresholds = np.unique(self.X[:, feature_index])

            for threshold in possible_thresholds:
                left_samples = []
                right_samples = []

                for sample in samples:
                    if sample[0][feature_index] <= threshold:
                        left_samples.append(sample)
                    else:
                        right_samples.append(sample)

                if not len(left_samples) or not len(right_samples):
                    continue

                left = Node(*map(np.array, zip(*left_samples)), self)
                right = Node(*map(np.array, zip(*right_samples)), self)

                information_gain = (self.gini - left.samples_count / self.samples_count * left.gini - 
                    right.samples_count / self.samples_count * right.gini)

                if information_gain > best_info_gain:
                    if self.parent and threshold == self.parent.left.threshold and feature_index == self.parent.left.feature_index:
                        continue
                    best_info_gain = information_gain
                    best_threshold = threshold
                    best_feature_index = feature_index
                    best_left = left
                    best_right = right
        
        self.feature_index = best_feature_index
        self.threshold = best_threshold
        self.left = best_left
        self.right = best_right

        return best_left, best_right


### Main class

This is the DecisionTreeClassifier class - the main class of the algorithm. It contains methods for building a decision tree out of any suitable dataset and a function for predicting classes for test samples, traversing a trained tree.

In [112]:
class DecisionTreeClassifier:
    """
    The main class of a decision tree classifier. Builds a decision tree and stores it as a single root node with
    its children as its instance variables.
    """

    def __init__(self, max_depth: int) -> None:
        self.max_depth = max_depth
        self.tree = None


    def fit(self, X: npt.NDArray, y: npt.NDArray) -> None:
        """
        Basically, function that performs all the training (building of a tree)
        We recommend to use it as a wrapper of recursive building function
        """
        root_node = Node(X, y)
        self.tree = root_node

        self.build_tree(root_node)

    def build_tree(self, node: Node, depth = 0) -> None:
        """
        Recursive building function
        """
        if depth < self.max_depth:
            left, right = node.find_best_split()

            if left and left.samples_count >= 2 and left.gini != 0:
                self.build_tree(left, depth + 1)

            if right and right.samples_count >= 2 and right.gini != 0:
                self.build_tree(right, depth + 1)
    
    def predict(self, X_test: npt.NDArray) -> npt.NDArray:
        """
        Traverse the tree while there is a child
        and return the predicted class for it 
        """
        if self.tree is None:
            raise ValueError("Cannot predict while the tree is not yet built. Use .fit()")

        predicted_classes = np.empty(X_test.shape[0], dtype=int)

        for index, sample in enumerate(list(X_test)):
            current_node = self.tree
            while current_node.threshold is not None:
                if sample[current_node.feature_index] <= current_node.threshold:
                    current_node = current_node.left
                else:
                    current_node = current_node.right

            predicted_classes[index] = max(zip(current_node.predicted_classes, current_node.class_occurences), key=lambda x: x[1])[0]

        return predicted_classes

    def evaluate(self, X_test: list[list], y_test: list) -> float:
        """
        Returns accuracy of the model (ratio of right guesses to the number of samples)
        """

        return float(np.sum(self.predict(X_test) == y_test) / y_test.shape[0])


### Building the decision tree

In [113]:
dtc = DecisionTreeClassifier(10)
dtc.fit(X, y)

### Evaluating the accuracy of the model

Now we can use our model to predict which type has a flower, basing on its parameters.

This is conducted basically via traversing the tree that you can see above.

In [ ]:
# Predict a testing dataset
predictions = dtc.predict(X_test)
print(predictions)
print(y_test)

In [ ]:
# Measure the accuracy
dtc.evaluate(X_test, y_test)

## Conclusion


In conclusion, we successfully built a decision tree classifier to predict the species of Iris flowers based on their features like sepal and petal length and width. By using the Iris dataset, we trained the model to split the data at each step based on the best features and values. Our aim was to create the most accurate predictions about the class each flower belongs to. The decision tree algorithm recursively built the tree, making predictions by following paths from the root to the leaves. With the performance of the algorithm we were able to classify new samples correctly, which showed us how decision trees can effectively classify data based on feature values.